## Data

In [10]:
# Import the drive and mount the remote directory
from google.colab import drive
drive.mount('/content/drive')

#Move to the directory
%cd /content/drive/MyDrive/ml_epfl/ml_road_segmentation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ml_epfl/ml_road_segmentation


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import random
import os, sys, time
from PIL import Image
import torchvision.transforms as T
import scipy.ndimage as sc
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import ImageFilter

In [12]:
#Constants
DIM_IMG = 400
DIM_IMG_CROP=DIM_IMG//2
NB_ROT = 6
ANGLE_ROTATION = 9
    
# Rotation
def rotation(imgs):
    rot_imgs = [T.functional.rotate(imgs[i], ANGLE_ROTATION*(idx_rot), expand=False)
                for idx_rot in range(NB_ROT)
                for i in range(len(imgs))
                ]
    return rot_imgs


def compose_all_functions_for_data(imgs):
    r = imgs + rotation(imgs)
    return r


def load_train_dataset():
    root_dir = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/data/training/"
    image_dir = root_dir + "images/"
    gt_dir = root_dir + "groundtruth/"
    files = os.listdir(image_dir)
    n = len(files)

    to_tensor = T.ToTensor()
    imgs = [to_tensor(Image.open(image_dir + files[i])) for i in range(n)]
    gt_imgs = [to_tensor(Image.open(gt_dir + files[i])) for i in range(n)]
    return (imgs, gt_imgs)

In [15]:
data = load_train_dataset()
training_imgs = compose_all_functions_for_data(data[0])
training_imgs_gt = compose_all_functions_for_data(data[1])

In [18]:
toPIL = T.ToPILImage()
training_imgs_PIL = [toPIL(img) for img in training_imgs]
training_imgs_gt_PIL = [toPIL(img) for img in training_imgs_gt]

for img_idx in range(len(training_imgs)):
    training_imgs_PIL[img_idx].save("/content/drive/MyDrive/data_abiola/training/training_" + '%.4d' % img_idx + ".png")
    training_imgs_gt_PIL[img_idx].save("/content/drive/MyDrive/data_abiola/training_gt/training_gt_" + '%.4d' % img_idx + ".png")